In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

import nltk
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords
import json

In [3]:
moviesDf = pd.read_csv('../data/tmdb_5000_movies.csv')
moviesDf.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
moviesDf.iloc[0]

budget                                                          237000000
genres                  [{"id": 28, "name": "Action"}, {"id": 12, "nam...
homepage                                      http://www.avatarmovie.com/
id                                                                  19995
keywords                [{"id": 1463, "name": "culture clash"}, {"id":...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [{"name": "Ingenious Film Partners", "id": 289...
production_countries    [{"iso_3166_1": "US", "name": "United States o...
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                               

#### Creating a string from all the features provided in the dataset.
> string = geners + keywords + original language + original title + overview + production companies + production countries + release date + spoken language + status + tagline + vote average + vote count

In [45]:
moviesDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   budget                   4803 non-null   int64  
 1   genres                   4803 non-null   object 
 2   homepage                 1712 non-null   object 
 3   id                       4803 non-null   int64  
 4   keywords                 4803 non-null   object 
 5   original_language        4803 non-null   object 
 6   original_title           4803 non-null   object 
 7   overview                 4800 non-null   object 
 8   popularity               4803 non-null   float64
 9   production_companies     4803 non-null   object 
 10  production_countries     4803 non-null   object 
 11  release_date             4802 non-null   object 
 12  revenue                  4803 non-null   int64  
 13  runtime                  4801 non-null   float64
 14  spoken_languages        

In [15]:
moviesDf.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


### Pre processing the data for generating the features required for generating the main feature

In [28]:
moviesDf['genresStr'] = moviesDf['genres'].apply(
    lambda genres: " ".join([gener.get('name') for gener in json.loads(genres)]))

In [30]:
moviesDf['keywordsStr'] = moviesDf['keywords'].apply(
    lambda keywords: " ".join([keyword.get('name') for keyword in json.loads(keywords)]))

In [33]:
moviesDf['production_companiesStr'] = moviesDf['production_companies'].apply(
    lambda production_companies : " ".join(production_company.get('name') for production_company in json.loads(production_companies)))

In [35]:
moviesDf['production_countriesStr'] = moviesDf['production_countries'].apply(
    lambda production_countries : " ".join(production_country.get('name') for production_country in json.loads(production_countries)))

In [37]:
moviesDf['spoken_languagesStr'] = moviesDf['spoken_languages'].apply(
    lambda spoken_languages: " ".join([spoken_language.get('name') for spoken_language in json.loads(spoken_languages)]))

In [39]:
moviesDf.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'genresStr', 'keywordsStr', 'production_companiesStr',
       'production_countriesStr', 'spoken_languagesStr'],
      dtype='object')

In [79]:
moviesDf['feature'] =  moviesDf['homepage'] \
                     + " " + moviesDf['original_language'] \
                     + " " + moviesDf['original_title'] \
                     + " " + moviesDf['overview'] \
                     + " " + moviesDf['popularity'].astype(str) \
                     + " " + moviesDf['release_date'].astype(str) \
                     + " " + moviesDf['revenue'].astype(str) \
                     + " " + moviesDf['runtime'].astype(str) \
                     + " " + moviesDf['status'] \
                     + " " + moviesDf['tagline'] \
                     + " " + moviesDf['title'] \
                     + " " + moviesDf['vote_average'].astype(str) \
                     + " " + moviesDf['vote_count'].astype(str) \
                     + " " + moviesDf['genresStr'] \
                     + " " + moviesDf['keywordsStr'] \
                     + " " + moviesDf['production_companiesStr'] \
                     + " " + moviesDf['production_countriesStr'] \
                     + " " + moviesDf['spoken_languagesStr']

In [74]:
moviesDf.fillna('', inplace=True)

### Implementing tfIdf for the data

In [81]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
moviesDf['feature'] = moviesDf['feature'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(moviesDf['feature'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(4803, 38283)

In [82]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [84]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(moviesDf.index, index=moviesDf['title']).drop_duplicates()

In [86]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return moviesDf['title'].iloc[movie_indices]

In [87]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
1359                                     Batman
428                              Batman Returns
119                               Batman Begins
3854    Batman: The Dark Knight Returns, Part 2
299                              Batman Forever
210                              Batman & Robin
9            Batman v Superman: Dawn of Justice
4267                                     Batman
303                                    Catwoman
Name: title, dtype: object

In [88]:
get_recommendations('The Avengers')

7                  Avengers: Age of Ultron
26              Captain America: Civil War
169     Captain America: The First Avenger
85     Captain America: The Winter Soldier
31                              Iron Man 3
182                                Ant-Man
79                              Iron Man 2
94                 Guardians of the Galaxy
126                   Thor: The Dark World
68                                Iron Man
Name: title, dtype: object

In [89]:
get_recommendations('Snowpiercer')

3840                                   Train
3516                               The Train
615                              Money Train
1518                      Dragon Wars: D-War
3492           The First Great Train Robbery
955                           The Peacemaker
545                              Unbreakable
1927                       Mrs. Winterbourne
3711             Sympathy for Lady Vengeance
1509    The Young and Prodigious T.S. Spivet
Name: title, dtype: object